In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/contractions/english_contractions.json
/kaggle/input/glove6b/glove.6B.50d.txt
/kaggle/input/glove6b/glove.6B.200d.txt
/kaggle/input/glove6b/glove.6B.100d.txt
/kaggle/input/glove6b/glove.6B.300d.txt


## Lectura de data sets

In [2]:
df_train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
df_test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

## Text preprocess

In [3]:
!pip install symspellpy

     |████████████████████████████████| 2.6 MB 605 kB/s eta 0:00:01
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [4]:
#Importamos librerias de pre procesamiento de texto
import nltk
import re
import string
from nltk.corpus import stopwords

import matplotlib as plt

from tqdm import tqdm
import spacy
from symspellpy.symspellpy import SymSpell, Verbosity
import pkg_resources
import json

In [5]:
#utilizamos la libreria "re" para limpieza de texto
#uso funcion 
#re.sub(pattern, repl, string, count=0, flags=0)
#elimino corchetes, links, <>, puntuaciones, saltos de linea
#y remuevo las palabras que contienen numeros

#remuevo todas las puntuaciones salvo # y @
#no remuevo urls pq los pienso reemplazar mas adelante
def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('<.*?>+', '', text)
    punctuations_to_remove = string.punctuation.replace("#","")
    punctuations_to_remove = punctuations_to_remove.replace("@","")
    text = re.sub('[%s]' % re.escape(punctuations_to_remove), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

#Remuevo emojis
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [6]:
def myStemmer(text):
    tokenizer = nltk.tokenize.TreebankWordTokenizer()
    tokens = tokenizer.tokenize(text)
    stemmer = nltk.stem.PorterStemmer()
    stemmed_list = []
    for token in tokens:
        stemmed_list.append(stemmer.stem(token))
    stemmed_text = ""
    separator = ' '
    stemmed_text = separator.join(stemmed_list)
    return stemmed_text

def myLemmatizer(text):
    tokenizer = nltk.tokenize.TreebankWordTokenizer()
    tokens = tokenizer.tokenize(text)
    lemmatizer = nltk.stem.WordNetLemmatizer()
    lemmatized_list = []
    for token in tokens:
        lemmatized_list.append(lemmatizer.lemmatize(token))
    lemmatized_text = ""
    separator = ' '
    lemmatized_text = separator.join(lemmatized_list)
    return lemmatized_text
    

In [7]:
def _lemmatize_text(sentence, nlp):
    sent = ""
    doc = nlp(sentence)
    for token in doc:
        if '@' in token.text:
            sent+=" @MENTION"
        elif '#' in token.text:
            sent+= " #HASHTAG"
        else:
            sent+=" "+token.lemma_
    return sent

In [8]:
def simplify(sentence):
    sent = _replace_urls(sentence)
    sent = _simplify_punctuation(sent)
    sent = _normalize_whitespace(sent)
    return sent

def _replace_urls(text):
    url_regex = r'(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})'
    text = re.sub(url_regex, "<URL>", text)
    return text

def _simplify_punctuation(text):
    """
    This function simplifies doubled or more complex punctuation. The exception is '...'.
    """
    corrected = str(text)
    corrected = re.sub(r'([!?,;])\1+', r'\1', corrected)
    corrected = re.sub(r'\.{2,}', r'...', corrected)
    return corrected

def _normalize_whitespace(text):
    """
    This function normalizes whitespaces, removing duplicates.
    """
    corrected = str(text)
    corrected = re.sub(r"//t",r"\t", corrected)
    corrected = re.sub(r"( )\1+",r"\1", corrected)
    corrected = re.sub(r"(\n)\1+",r"\1", corrected)
    corrected = re.sub(r"(\r)\1+",r"\1", corrected)
    corrected = re.sub(r"(\t)\1+",r"\1", corrected)
    return corrected.strip(" ")

In [9]:
def normalize_contractions(text, contractions):
    """
    This function normalizes english contractions.
    """
    new_token_list = []
    token_list = text.split()
    for word_pos in range(len(token_list)):
        word = token_list[word_pos]
        first_upper = False
        if word[0].isupper():
            first_upper = True
        if word.lower() in contractions:
            replacement = contractions[word.lower()]
            if first_upper:
                replacement = replacement[0].upper()+replacement[1:]
            replacement_tokens = replacement.split()
            if len(replacement_tokens)>1:
                new_token_list.append(replacement_tokens[0])
                new_token_list.append(replacement_tokens[1])
            else:
                new_token_list.append(replacement_tokens[0])
        else:
            new_token_list.append(word)
    sentence = " ".join(new_token_list).strip(" ")
    return sentence

In [10]:
def init_spellchecker():
    max_edit_distance_dictionary= 3
    prefix_length = 4
    spellchecker = SymSpell(max_edit_distance_dictionary, prefix_length)
    dictionary_path = pkg_resources.resource_filename(
        "symspellpy", "frequency_dictionary_en_82_765.txt")
    bigram_path = pkg_resources.resource_filename(
        "symspellpy", "frequency_bigramdictionary_en_243_342.txt")
    spellchecker.load_dictionary(dictionary_path, term_index=0, count_index=1)
    spellchecker.load_bigram_dictionary(dictionary_path, term_index=0, count_index=2)
    return spellchecker
    
def spell_correction(text, spellchecker):
    """
    This function does very simple spell correction normalization using pyspellchecker module. It works over a tokenized sentence and only the token representations are changed.
    """
    if len(text) < 1:
        return ""
    #Spell checker config
    max_edit_distance_lookup = 2
    suggestion_verbosity = Verbosity.TOP # TOP, CLOSEST, ALL
    #End of Spell checker config
    token_list = text.split()
    for word_pos in range(len(token_list)):
        word = token_list[word_pos]
        if word is None:
            token_list[word_pos] = ""
            continue
        if not '\n' in word and word not in string.punctuation and not is_numeric(word) and not (word.lower() in spellchecker.words.keys()):
            suggestions = spellchecker.lookup(word.lower(), suggestion_verbosity, max_edit_distance_lookup)
            #Checks first uppercase to conserve the case.
            upperfirst = word[0].isupper()
            #Checks for correction suggestions.
            if len(suggestions) > 0:
                correction = suggestions[0].term
                replacement = correction
            #We call our _reduce_exaggerations function if no suggestion is found. Maybe there are repeated chars.
            else:
                replacement = _reduce_exaggerations(word)
            #Takes the case back to the word.
            if upperfirst:
                replacement = replacement[0].upper()+replacement[1:]
            word = replacement
            token_list[word_pos] = word
    return " ".join(token_list).strip()

def _reduce_exaggerations(text):
    """
    Auxiliary function to help with exxagerated words.
    Examples:
        woooooords -> words
        yaaaaaaaaaaaaaaay -> yay
    """
    correction = str(text)
    #TODO work on complexity reduction.
    return re.sub(r'([\w])\1+', r'\1', correction)

def is_numeric(text):
    for char in text:
        if not (char in "0123456789" or char in ",%.$"):
            return False
    return True


In [11]:
def text_preprocessing(text, nlp, contractions, spellchecker):
    text = simplify(text)
    text = normalize_contractions(text, contractions)
    text = spell_correction(text, spellchecker)
    #la limpieza la hago al final dado 
    #que remueve todas las puntuaciones y muchas son importantes
    #para la normalizacion del texto como (')
    text = clean_text(text)
    text = myStemmer(text)
    #text = myLemmatizer(text)
    #text = _lemmatize_text(text, nlp).strip()
    return text

In [12]:
nlp = spacy.load('en')
spellchecker = init_spellchecker()
contraction_list = json.loads(open('../input/contractions/english_contractions.json', 'r').read())    
x = "Our Deeds are the Reason of this #earthquake"
print(text_preprocessing(x, nlp, contraction_list, spellchecker))

our deed are the reason of thi earthquak


## Agrego mas features

In [13]:
# word_count
df_train['word_count'] = df_train['text'].apply(lambda x: len(str(x).split()))
df_test['word_count'] = df_test['text'].apply(lambda x: len(str(x).split()))

# unique_word_count
df_train['unique_word_count'] = df_train['text'].apply(lambda x: len(set(str(x).split())))
df_test['unique_word_count'] = df_test['text'].apply(lambda x: len(set(str(x).split())))

# stop_word_count
df_train['stop_word_count'] = df_train['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in stopwords.words('english')]))
df_test['stop_word_count'] = df_test['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in stopwords.words('english')]))

# url_count
df_train['url_count'] = df_train['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))
df_test['url_count'] = df_test['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))

# mean_word_length
df_train['mean_word_length'] = df_train['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
df_test['mean_word_length'] = df_test['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

# char_count
df_train['char_count'] = df_train['text'].apply(lambda x: len(str(x)))
df_test['char_count'] = df_test['text'].apply(lambda x: len(str(x)))

# punctuation_count
df_train['punctuation_count'] = df_train['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
df_test['punctuation_count'] = df_test['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))

# hashtag_count
df_train['hashtag_count'] = df_train['text'].apply(lambda x: len([c for c in str(x) if c == '#']))
df_test['hashtag_count'] = df_test['text'].apply(lambda x: len([c for c in str(x) if c == '#']))

# mention_count
df_train['mention_count'] = df_train['text'].apply(lambda x: len([c for c in str(x) if c == '@']))
df_test['mention_count'] = df_test['text'].apply(lambda x: len([c for c in str(x) if c == '@']))

### Aplico funciones de preprocesamiento al data set

In [14]:
#pre- procesamiento
df_train.head(10)

,id,keyword,location,text,target,word_count,unique_word_count,stop_word_count,url_count,mean_word_length,char_count,punctuation_count,hashtag_count,mention_count
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,13,13,6,0,4.384615,69,1,1,0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,7,7,0,0,4.571429,38,1,0,0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,22,20,11,0,5.090909,133,3,0,0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,8,8,1,0,7.125000,65,2,1,0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,16,15,7,0,4.500000,88,2,2,0
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1,18,18,3,0,5.166667,110,7,3,0
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1,14,14,2,0,5.857143,95,3,2,0
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1,15,14,9,0,3.000000,59,4,0,0
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1,12,11,5,0,5.666667,79,1,0,0
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1,10,10,5,0,4.300000,52,4,0,0


In [15]:
df_train['text'][5]

'#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires'

In [16]:
nlp = spacy.load('en')
spellchecker = init_spellchecker()
contraction_list = json.loads(open('../input/contractions/english_contractions.json', 'r').read())    
#Train set
df_train['text'] = df_train['text']\
    .apply(lambda x: text_preprocessing(x, nlp, contraction_list, spellchecker))
#Test set
df_test['text'] = df_test['text']\
    .apply(lambda x: text_preprocessing(x, nlp, contraction_list, spellchecker))

In [17]:
df_train['text'][5]

'# rockyfir updat california hwi close in both direct due to lake counti fire afir wildfir'

In [18]:
#post- procesamiento
df_train.head()

,id,keyword,location,text,target,word_count,unique_word_count,stop_word_count,url_count,mean_word_length,char_count,punctuation_count,hashtag_count,mention_count
0,1,NaN,NaN,our deed are the reason of thi earthquak may a...,1,13,13,6,0,4.384615,69,1,1,0
1,4,NaN,NaN,forest fire near la rang sask canada,1,7,7,0,0,4.571429,38,1,0,0
2,5,NaN,NaN,all resid ask to shelter in place are be notif...,1,22,20,11,0,5.090909,133,3,0,0
3,6,NaN,NaN,peopl receiv wildfir evacu order in california,1,8,8,1,0,7.125000,65,2,1,0
4,7,NaN,NaN,just got sent thi photo from rubi alaska as sm...,1,16,15,7,0,4.500000,88,2,2,0


In [19]:
df_test.head()

,id,keyword,location,text,word_count,unique_word_count,stop_word_count,url_count,mean_word_length,char_count,punctuation_count,hashtag_count,mention_count
0,0,NaN,NaN,just happen a terribl car crash,6,6,2,0,4.833333,34,0,0,0
1,2,NaN,NaN,heard about earthquak is differ citi stay safe...,9,9,2,0,6.222222,64,3,1,0
2,3,NaN,NaN,there is a forest fire at spot pond gees are f...,19,19,9,0,4.105263,96,2,0,0
3,9,NaN,NaN,apocalyps light spokan wildfir,4,4,0,0,9.250000,40,3,2,0
4,11,NaN,NaN,typhoon soudelor kill in china and taiwan,8,8,2,0,4.750000,45,0,0,0


### (Opcional) Agrego los features pero despues de la limpieza

## Features extraction

In [20]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion

In [21]:
#Feature extraction - BOW
count_vectorizer = CountVectorizer(ngram_range=(1,2))
train_bow = count_vectorizer.fit_transform(df_train['text'])
#print(count_vectorizer.get_feature_names())
test_bow = count_vectorizer.transform(df_test['text'])

['aah', 'aah ani', 'aampb', 'aampb pipelin', 'aampw', 'aampw from', 'aand', 'aand credit', 'aand there', 'aaronthefm', 'aaronthefm guy', 'aashiqui', 'aashiqui actress', 'ab', 'ab url', 'aba', 'aba adida', 'aba as', 'aba url', 'abadan', 'abadan they', 'abandon', 'abandon aircraft', 'abandon at', 'abandon cabin', 'abandon cocker', 'abandon nor', 'abandon plan', 'abandon ship', 'abandon the', 'abandonedp', 'abandonedp you', 'abbi', 'abbi can', 'abbi news', 'abbi to', 'abbott', 'abbott camp', 'abbott must', 'abc', 'abc helicopt', 'abc of', 'abc onlin', 'abcnew', 'abcnew nuclear', 'abcnew obama', 'abcnew us', 'abcnorio', 'abcnorio garden', 'abe', 'abe delug', 'abe govern', 'abe pledg', 'aberdeen', 'aberdeen tomorrow', 'aberdenfanpag', 'aberdenfanpag good', 'aberdenfc', 'aberdenfc aberdenfanpag', 'aberystwythshrewsburi', 'aberystwythshrewsburi right', 'abet', 'abet govern', 'abey', 'abey nicolekidman', 'abha', 'abha fatal', 'abil', 'abil to', 'abject', 'abject desol', 'abl', 'abl to', 'ablaz

In [22]:
#TF IDF
tfidf_vectorizer = TfidfVectorizer()
train_tfidf = tfidf_vectorizer.fit_transform(df_train['text'])
#print(tfidf_vectorizer.get_feature_names())
test_tfidf = tfidf_vectorizer.transform(df_test['text'])

['aah', 'aampb', 'aampw', 'aand', 'aaronthefm', 'aashiqui', 'ab', 'aba', 'abadan', 'abandon', 'abandonedp', 'abbi', 'abbott', 'abc', 'abcnew', 'abcnorio', 'abe', 'aberdeen', 'aberdenfanpag', 'aberdenfc', 'aberystwythshrewsburi', 'abet', 'abey', 'abha', 'abil', 'abject', 'abl', 'ablaz', 'abninfvet', 'aboard', 'abomin', 'abort', 'abot', 'about', 'abov', 'abram', 'abrancabalero', 'abridg', 'abruchsimul', 'absenc', 'absolut', 'abstorm', 'abstract', 'absurd', 'absurdli', 'abswinston', 'abus', 'abuseddesolateamplost', 'abyairshow', 'abysmaljoin', 'acaciapen', 'academia', 'acarewornheart', 'acc', 'accept', 'access', 'accid', 'accident', 'accompani', 'accord', 'accordingli', 'account', 'accuraci', 'accus', 'accustom', 'acdelco', 'ace', 'acebab', 'acebreakingnew', 'acenewsdesk', 'aceorg', 'acept', 'ach', 'achedin', 'achiev', 'achimota', 'achoo', 'acid', 'acidentalpropheci', 'acidents', 'acidentwho', 'acids', 'acionempresa', 'acn', 'acoust', 'acousticmaloley', 'acquiesc', 'acquir', 'acquisit', '

## XGBoost classifier

In [23]:
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import DMatrix

In [24]:
#clf = XGBClassifier(random_state=42, seed=2, colsample_bytree=0.6, subsample=0.7)
clf = XGBClassifier(objective = 'binary:logistic', 
                    random_state=42, 
                    seed=2, 
                    colsample_bytree=0.5, 
                    subsample=0.7,
                    learning_rate=0.1,
                    )

## Crear Pipeline de features
Dado que tenemos uan combinacion de features numericos y de texto, uso funciones "helpers" que me devuelven la columna del texto o la columna del numero. Esto me permite alimentar el algoritmo con dos tipos de features distintos.

Since we have combined input features, text and numeric based features, we cannot treat them in the same way. We create helper classes that will return either the text column (TextSelector) or the number based column (NumberSelector). They will be the part of the pipeline. You initialize them with the column name of the input data. They need to have a fit and transform method. In our case the fit returns the class itself, while the transform method returns the column of the input data with the given column name.

In [25]:
from sklearn.base import BaseEstimator, TransformerMixin
class TextSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on text columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None, *parg, **kwarg):
        return self

    def transform(self, X):
        # returns the input as a string
        return X[self.key]

class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # returns the input as a dataframe
        return X[[self.key]]

In [26]:
text = Pipeline([
    ('selector', TextSelector(key='text')),
    ('vectorizer', count_vectorizer)])

word_count = Pipeline([
    ('selector',NumberSelector(key='word_count'))
])

unique_word_count = Pipeline([
    ('selector',NumberSelector(key='unique_word_count'))
])

stop_word_count = Pipeline([
    ('selector',NumberSelector(key='unique_word_count'))
])

url_count = Pipeline([
    ('selector',NumberSelector(key='url_count'))
])

mean_word_length = Pipeline([
    ('selector',NumberSelector(key='mean_word_length'))
])

char_count = Pipeline([
    ('selector',NumberSelector(key='char_count'))
])

punctuation_count = Pipeline([
    ('selector',NumberSelector(key='punctuation_count'))
])

hashtag_count = Pipeline([
    ('selector',NumberSelector(key='hashtag_count'))
])

mention_count = Pipeline([
    ('selector',NumberSelector(key='mention_count'))
])

In [27]:
features = FeatureUnion([('text', text),
                        #('word_count', word_count),
                        ('unique_word_count', unique_word_count),
                        ('stop_word_count', stop_word_count),
                        #('url_count', url_count),
                        ('mean_word_length', mean_word_length),
                        #('char_count', char_count),
                        #('punctuation_count', punctuation_count),
                        #('hashtag_count', hashtag_count),
                        ('mention_count', mention_count)
                        ])

In [28]:
pipe = Pipeline([('features', features),
                ('clf', clf)
                ])

In [29]:
pipe.named_steps['clf'].get_params()

{'objective': 'binary:logistic',
 'base_score': None,
 'booster': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': 0.5,
 'gamma': None,
 'gpu_id': None,
 'importance_type': 'gain',
 'interaction_constraints': None,
 'learning_rate': 0.1,
 'max_delta_step': None,
 'max_depth': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'n_estimators': 100,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': 42,
 'reg_alpha': None,
 'reg_lambda': None,
 'scale_pos_weight': None,
 'subsample': 0.7,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None,
 'seed': 2}

## Split train y validation set

In [30]:
from sklearn.model_selection import train_test_split, GridSearchCV

In [31]:
combined_features = ['text', 
                    #'word_count', 
                    'unique_word_count',
                    'stop_word_count', 
                    #'url_count',
                    'mean_word_length',
                    #'char_count', 
                    #'punctuation_count', 
                    #'hashtag_count',
                    'mention_count'
                    ]
target = 'target'

X_train, X_test, y_train, y_test = train_test_split(df_train[combined_features], 
                                                    df_train[target], 
                                                    test_size=0.33, random_state=42, 
                                                    #stratify=df_train[target]
                                                     )
print(X_train)
print(y_train)
#print(X_test)
#print(y_test)

                                                   text  unique_word_count  \
5836  photo postapocalypticflimflam prod around the ...                  7   
30                                              the end                  2   
1879            man crush everyday @ cristianinspir url                  6   
6852  @ ptsdchat ye i feel the root of that is shame...                 20   
2673  @ autoam everyon hope we would join isi and ge...                 24   
...                                                 ...                ...   
5226  @ there are not mani obliter server but i alwa...                 16   
5390  just had a panic attack by i do not have enoug...                 27   
860   moron automat blood pressur monitor standard a...                 13   
7603  offici say a quarantin is in place at an alaba...                 19   
7270  i move to england five year ago today what a w...                 17   

      stop_word_count  mean_word_length  mention_count  
5836  

## Tuneo hiperparametros

In [32]:
param_grid = {
     'clf__n_estimators': [50,100,300],
  #  'clf__colsample_bytree': [0.6,0.8,1]
#    'clf__subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1]
}

# grid search cross validation instantiation
grid_search = GridSearchCV(estimator = pipe, param_grid = param_grid, 
                          cv = 5, n_jobs = 1, verbose = 0, return_train_score=True)

In [33]:
#hyperparameter fitting
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text',
                                                                        Pipeline(steps=[('selector',
                                                                                         TextSelector(key='text')),
                                                                                        ('vectorizer',
                                                                                         CountVectorizer(ngram_range=(1,
                                                                                                                      2)))])),
                                                                       ('unique_word_count',
                                                                        Pipeline(steps=[('selector',
                                                                                       

In [34]:
grid_search.cv_results_['mean_train_score']

array([0.79598039, 0.82848039, 0.88593137])

In [35]:
grid_search.cv_results_['mean_test_score']

array([0.75117647, 0.76215686, 0.78039216])

In [36]:
grid_search.best_params_

{'clf__n_estimators': 300}

## Prediccion train y validacion
#### score validacion (acertados para 2513 de validacion)

Todos tienen text
- tf-idf: 1939
- tf-idf + word_count: 1961
- tf_idf + todo: 1945
- bow: 1991
- bow + ngram(1,2) : 1992
- bow(ngram(1,2)) + word_count : 1991
- bow(ngram(1,2)) + word_count + unique_word_count : 1982
- bow(ngram(1,2)) + unique_word_count : 1996
- bow(ngram(1,2)) + unique_word_count + stop_words_count : 2004
- bow(ngram(1,2)) + unique_word_count + stop_words_count + url_count : 2000
- bow(ngram(1,2)) + unique_word_count + stop_words_count + mean_word_length : 2005
- bow(ngram(1,2)) + unique_word_count + stop_words_count + mean_word_length + char_count : 1994
- bow(ngram(1,2)) + unique_word_count + stop_words_count + mean_word_length + punctuation_count : 2005
- bow(ngram(1,2)) + unique_word_count + stop_words_count + mean_word_length + hashtag_count: 1996
- bow(ngram(1,2)) + unique_word_count + stop_words_count + mean_word_length + mention_count: 2006

In [37]:
clf_test = grid_search.best_estimator_

In [38]:
pred_test = clf_test.predict(X_test)
#pred_test = clf.predict(X_test)

In [39]:
#Hago un df con las prediciones y los y_test para compararlos
dic_compare_pred_test = {'pred_test':pred_test, 'y_test':y_test.values}

In [40]:
df_compare_pred_test = pd.DataFrame(data=dic_compare_pred_test)

In [41]:
df_compare_pred_test

,pred_test,y_test
0,0,1
1,0,0
2,1,1
3,0,0
4,0,0
...,...,...
2508,0,0
2509,0,1
2510,1,1
2511,1,1


In [42]:
df_compare_pred_test['correct'] = (df_compare_pred_test['pred_test'] == df_compare_pred_test['y_test'])

In [43]:
df_compare_pred_test.head()

,pred_test,y_test,correct
0,0,1,False
1,0,0,True
2,1,1,True
3,0,0,True
4,0,0,True


In [44]:
df_compare_pred_test['correct'].value_counts()

True     2005
False     508
Name: correct, dtype: int64

### Hiperparametros
Solo texto como feature
Los n_estimators no alteran el puntaje pq los defino con el gridSearchCV best_params
- random_state=42, seed=2, colsample_bytree=0.2, subsample=0.7: 1959
- random_state=42, seed=2, colsample_bytree=0.2, subsample=0.2: 1833
- random_state=42, seed=2, colsample_bytree=0.2, subsample=0.7, n_estimators=1000 : 1959
-  random_state=42, seed=2, colsample_bytree=0.2, subsample=0.7, n_estimators=1000, learning_rate=0.1 : 1973
- random_state=42, seed=2, colsample_bytree=0.5, subsample=0.7, n_estimators=1000, learning_rate=0.1 : **1982**
- random_state=42, seed=2, colsample_bytree=0.5, subsample=0.7, n_estimators=1000, learning_rate=0.5 : 1893
- random_state=42, seed=2, colsample_bytree=0.5, subsample=0.7, n_estimators=1000, learning_rate=0.1, gamma=0.2 : 1982
- random_state=42, seed=2, colsample_bytree=0.5, subsample=0.7, n_estimators=1000, learning_rate=0.1, gamma=0.2, alpha=10 : 1836
- random_state=42, seed=2, colsample_bytree=0.5, subsample=0.7, n_estimators=1000, learning_rate=0.1, gamma=0.2, alpha=100 : 1699
- random_state=42, seed=2, colsample_bytree=0.5, subsample=0.7, n_estimators=1000, learning_rate=0.1, gamma=0.2, alpha=1 : 1953
- random_state=42, seed=2, colsample_bytree=0.5, subsample=0.7, n_estimators=1000, learning_rate=0.1, gamma=0.2, alpha=0.5 : 1961
- random_state=42, seed=2, colsample_bytree=0.5, subsample=0.7, n_estimators=1000, learning_rate=0.1, gamma=0.8 : 1978
- random_state=42, seed=2, colsample_bytree=0.5, subsample=0.7, n_estimators=1000, learning_rate=0.1, gamma=10 : 1892
- random_state=42, seed=2, colsample_bytree=0.5, subsample=0.7, n_estimators=1000, learning_rate=0.1, gamma=1 : 1977
- random_state=42, seed=2, colsample_bytree=0.5, subsample=0.7, learning_rate=0.1, max_depth=15 : 1948
-  random_state=42, seed=2, colsample_bytree=0.5, subsample=0.7, learning_rate=0.1, max_depth=5 : 1971
- random_state=42, seed=2, colsample_bytree=0.5, subsample=0.7, learning_rate=0.1, max_depth=50 : 1929
- random_state=42, seed=2, colsample_bytree=0.5, subsample=0.7, learning_rate=0.1, min_child_weight=3 : 1920
- random_state=42, seed=2, colsample_bytree=0.5, subsample=0.7, learning_rate=0.1, min_child_weight=10 : 1843
- random_state=42, seed=2, colsample_bytree=0.5, subsample=0.7, learning_rate=0.02 : 1899
- random_state=42, seed=2, colsample_bytree=0.5, subsample=0.7, learning_rate=0.2 : 1971

#### bow(n_gram(1,2) 
- random_state=42, seed=2, colsample_bytree=0.5, subsample=0.7, learning_rate=0.01: 1852
- colsample_bytree=0.5, subsample=0.7, learning_rate=0.1 : 1992
- random_state=42, seed=2, colsample_bytree=0.5, subsample=0.7, learning_rate=0.1, max_depth=1 : 1902
-  random_state=42, seed=2, colsample_bytree=0.5, subsample=0.7, learning_rate=0.1 : **2006**
                 
                   

## Predict

Despues de validarlo ahora hacemos la prediccion con el test set

In [45]:
#obtengo los features del test set
data = df_test[combined_features]
data

,text,unique_word_count,stop_word_count,mean_word_length,mention_count
0,just happen a terribl car crash,6,2,4.833333,0
1,heard about earthquak is differ citi stay safe...,9,2,6.222222,0
2,there is a forest fire at spot pond gees are f...,19,9,4.105263,0
3,apocalyps light spokan wildfir,4,0,9.250000,0
4,typhoon soudelor kill in china and taiwan,8,2,4.750000,0
...,...,...,...,...,...
3258,earthquak safeti lo angel ûò safeti fasten own,7,0,6.000000,0
3259,storm in i wors than last hurrican my hardest ...,22,6,5.086957,0
3260,green line derail in chicago url,6,1,8.333333,0
3261,meg issu hazard weather outlook hwo url,7,0,8.428571,0


In [46]:
pred = clf_test.predict(data)

In [47]:
pred

array([0, 1, 1, ..., 1, 1, 0])

## Submission

In [48]:
sample_submission = pd.read_csv("../input/nlp-getting-started/sample_submission.csv")

In [49]:
sample_submission['target'] = pred
sample_submission.to_csv("submission.csv", index=False)
sample_submission.head(10)

,id,target
0,0,0
1,2,1
2,3,1
3,9,1
4,11,1
5,12,1
6,21,0
7,22,0
8,27,0
9,29,0


In [50]:
sample_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   id      3263 non-null   int64
 1   target  3263 non-null   int64
dtypes: int64(2)
memory usage: 51.1 KB


## Verifico submission.csv

In [51]:
df_sub = pd.read_csv("./submission.csv")
df_test_copy = df_test.copy(deep=True)
df_test_copy.head()

,id,keyword,location,text,word_count,unique_word_count,stop_word_count,url_count,mean_word_length,char_count,punctuation_count,hashtag_count,mention_count
0,0,NaN,NaN,just happen a terribl car crash,6,6,2,0,4.833333,34,0,0,0
1,2,NaN,NaN,heard about earthquak is differ citi stay safe...,9,9,2,0,6.222222,64,3,1,0
2,3,NaN,NaN,there is a forest fire at spot pond gees are f...,19,19,9,0,4.105263,96,2,0,0
3,9,NaN,NaN,apocalyps light spokan wildfir,4,4,0,0,9.250000,40,3,2,0
4,11,NaN,NaN,typhoon soudelor kill in china and taiwan,8,8,2,0,4.750000,45,0,0,0


In [52]:
#Copio el target de la prediccion al data set de test
df_test_copy['target'] = df_sub['target']
df_test_copy.head(60)

,id,keyword,location,text,word_count,unique_word_count,stop_word_count,url_count,mean_word_length,char_count,punctuation_count,hashtag_count,mention_count,target
0,0,NaN,NaN,just happen a terribl car crash,6,6,2,0,4.833333,34,0,0,0,0
1,2,NaN,NaN,heard about earthquak is differ citi stay safe...,9,9,2,0,6.222222,64,3,1,0,1
2,3,NaN,NaN,there is a forest fire at spot pond gees are f...,19,19,9,0,4.105263,96,2,0,0,1
3,9,NaN,NaN,apocalyps light spokan wildfir,4,4,0,0,9.250000,40,3,2,0,1
4,11,NaN,NaN,typhoon soudelor kill in china and taiwan,8,8,2,0,4.750000,45,0,0,0,1
5,12,NaN,NaN,we are shakingit an earthquak,4,4,1,0,7.750000,34,5,0,0,1
6,21,NaN,NaN,they had probabl still show more life than ars...,12,12,3,0,5.083333,72,4,0,0,0
7,22,NaN,NaN,hey how are you,4,4,2,0,3.500000,17,2,0,0,0
8,27,NaN,NaN,what a nice hat,4,4,2,0,3.250000,16,1,0,0,0
9,29,NaN,NaN,fuck off,2,2,0,0,4.000000,9,1,0,0,0
